In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [2]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

INFO - 2020-11-29 02:58:51,139 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-29 02:58:51,140 - settings - Setting database.user to celiib
INFO - 2020-11-29 02:58:51,141 - settings - Setting database.password to newceliipass


In [3]:
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-29 02:58:51,156 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-29 02:58:51,157 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-29 02:58:51,170 - connection - Connected celiib@at-database.ad.bcm.edu:3306


Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-29 02:58:51,373 - settings - Setting enable_python_native_blobs to True


# Creating the Exclusion Criteria Table

In [4]:
import datajoint as dj
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
pinky.ExclusionCriteria()

criteria_id key by which to lookup the exclusion criteria description,criteria_desc criteria by which this segment was excluded (preferred to add name of excluder as well)
0,Labeling pipeline
1,postsyn label was error in labeling pipeline
2,All error segments


In [ ]:
pinky.ExclusionCriteria.describe()

In [ ]:
@schema
class ExclusionCriteria(dj.Manual):
    definition="""
    # Descriptions of exclusion criterias
    criteria_id          : int unsigned                 # key by which to lookup the exclusion criteria description
    ---
    criteria_desc        : varchar(64)                  # criteria by which this segment was excluded (preferred to add name of excluder as well)
    
    """

In [ ]:
ExclusionCriteria.insert1(dict(criteria_id=0,criteria_desc="Axon Cleaning 11/27"))

In [ ]:
ExclusionCriteria()

# Creating the Synapse Exclude

In [5]:
pinky.SynapseExclude.describe()

-> pinky.Synapse
-> pinky.ExclusionCriteria



'-> pinky.Synapse\n-> pinky.ExclusionCriteria\n'

In [7]:
minnie.Segment.describe()

# Segment: a volumetric segmented object
segment_id           : bigint unsigned              # segment id unique within each Segmentation



'# Segment: a volumetric segmented object\nsegment_id           : bigint unsigned              # segment id unique within each Segmentation\n'

In [14]:
@schema
class SynapseExclude(dj.Manual):
    definition="""
    # Descriptions of exclusion criterias
    -> minnie.Synapse
    -> minnie.ExclusionCriteria
    ---
    segment_id           : bigint unsigned              # segment id that caused the synapse to be error
    """

In [16]:
SynapseExclude()#.drop()

synapse_id synapse index within the segmentation,timestamp,criteria_id key by which to lookup the exclusion criteria description,segment_id segment id that caused the synapse to be error


# The labels table that will store all of the error labels

In [ ]:
pinky.CompartmentFinal.describe()

In [ ]:
minnie.Decomposition()

In [ ]:
pinky.ProofreadLabel.describe()

In [26]:
@schema
class AutoProofreadLabels(dj.Computed):
    definition="""
    -> minnie.Decomposition
    ---
    n_face_errors : int #the number of faces that were errored out
    face_idx_for_error : longblob #the face indices for the errors computed
    n_synapses: smallint unsigned #total number of synpases
    n_errored_synapses: smallint unsigned #the number of synapses
    """
    

In [29]:
AutoProofreadLabels()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_face_errors the number of faces that were errored out,face_idx_for_error the face indices for the errors computed,n_synapses total number of synpases,n_errored_synapses the number of synapses
864691134884741370,0,0.25,208443,=BLOB=,469,207
864691134884742138,0,0.25,13216,=BLOB=,214,19
864691134884742906,0,0.25,325377,=BLOB=,477,181
864691134884743418,0,0.25,81085,=BLOB=,227,59
864691134884743930,0,0.25,119620,=BLOB=,303,68


In [19]:
minnie.DecompositionErrorLabels()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_face_errors the number of faces that were errored out,face_idx_for_error the face indices for the errors computed
864691134884740858,0,0.25,159052,=BLOB=
864691134884741114,0,0.25,40238,=BLOB=
864691134884741370,0,0.25,208443,=BLOB=
864691134884743418,0,0.25,81085,=BLOB=
864691134884743930,0,0.25,119620,=BLOB=
864691134884744186,0,0.25,139734,=BLOB=
864691134884744442,0,0.25,3345139,=BLOB=
864691134884745210,0,0.25,25349,=BLOB=
864691134884747002,0,0.25,102810,=BLOB=
864691134884748026,0,0.25,88201,=BLOB=


In [23]:
segment_id = 864691134884749050


using quick fetch


In [24]:
returned_error_faces

array([  1469,   1470,   1471, ..., 570146, 570147, 570148])